In [1]:
import pandas as pd

# 1. LOAD DATA (First 50,000 rows only)

df = pd.read_csv('US_Accidents_March23.csv', nrows=50000)
print("Data loaded successfully!")

# 2. EXPLORE STRUCTURE
print(f"Dataset Shape: {df.shape}")  # (Rows, Columns)
print(f"Columns: {df.columns.tolist()}")

# 3. CHECK DATA TYPES
print("\n--- Data Info ---")
df.info()

# 4. CHECK MISSING VALUES
print("\n--- Missing Values (Top 10) ---")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# 5. BASIC STATISTICS
print("\n--- Basic Statistics ---")
display(df.describe()) # 'display' looks better in Jupyter than 'print'


Data loaded successfully!
Dataset Shape: (50000, 46)
Columns: ['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     50000 non-null  object 
 1  

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,50000.00000,50000.000000,50000.000000,0.0,0.0,50000.000000,49544.000000,2497.000000,49361.000000,49694.000000,49474.000000,41578.000000,3276.000000
mean,2.39678,37.396862,-120.736826,NaN,NaN,0.014660,64.975323,39.050541,61.719394,29.981206,9.384875,8.736789,0.026529
std,0.49225,1.553001,4.750326,NaN,NaN,0.354411,13.200795,14.914320,21.706796,0.195356,1.958316,4.475498,0.050507
min,1.00000,32.544304,-123.813927,NaN,NaN,0.000000,3.900000,-8.400000,4.000000,3.010000,0.100000,0.000000,0.000000
25%,2.00000,37.343647,-122.127098,NaN,NaN,0.000000,55.900000,33.400000,46.000000,29.900000,10.000000,5.800000,0.000000
50%,2.00000,37.776711,-121.828537,NaN,NaN,0.000000,64.400000,39.000000,63.000000,29.970000,10.000000,8.100000,0.010000
75%,3.00000,38.409191,-121.272102,NaN,NaN,0.010000,73.400000,42.500000,78.000000,30.070000,10.000000,11.500000,0.030000
max,4.00000,41.428753,-81.550728,NaN,NaN,51.130000,113.000000,101.000000,100.000000,33.040000,80.000000,142.700000,0.550000


In [ ]:
# ==========================================
# WEEK 2: DATA CLEANING AND PREPROCESSING
# ==========================================

# 1. HANDLE MISSING VALUES
# ------------------------------------------
# Requirement: Drop or impute columns with excessive missing values 
# Based on your Week 1 findings:
# - 'End_Lat' and 'End_Lng' were empty (0 non-null).
# - 'Precipitation(in)' had very few values.

print("--- Cleaning Data ---")

# A. Drop columns with too much missing data
# We drop End_Lat/Lng because they are empty. 
# We keep Wind_Chill for now but you could drop it if it's >90% empty in larger sets.
cols_to_drop = ['End_Lat', 'End_Lng']
df_clean = df.drop(columns=cols_to_drop)
print(f"Dropped columns: {cols_to_drop}")

# B. Impute (Fill) missing values
# For precipitation, we assume NaN means 0 (no rain).
df_clean['Precipitation(in)'] = df_clean['Precipitation(in)'].fillna(0)
print("Filled missing Precipitation values with 0.")

# C. Drop rows with missing critical data
# We cannot analyze an accident if we don't know the City or Street.
df_clean = df_clean.dropna(subset=['Street', 'City', 'Zipcode'])
print("Dropped rows with missing City/Street info.")


# 2. CONVERT DATETIME COLUMNS
# ------------------------------------------
# Requirement: Convert datetime columns into proper formats 
print("\n--- Converting Dates ---")

# 'errors=coerce' turns unreadable dates into NaT (Not a Time) so they don't crash the code
df_clean['Start_Time'] = pd.to_datetime(df_clean['Start_Time'], errors='coerce')
df_clean['End_Time'] = pd.to_datetime(df_clean['End_Time'], errors='coerce')

print("Start_Time and End_Time converted to datetime objects.")


# 3. FEATURE ENGINEERING
# Requirement: Create new features like "Hour", "Weekday", or "Month" 
print("\n--- Creating New Features ---")

# Extract the Hour (0-23)
df_clean['Hour'] = df_clean['Start_Time'].dt.hour

# Extract the Month (1-12)
df_clean['Month'] = df_clean['Start_Time'].dt.month

# Extract the Day of the Week (Monday, Tuesday, etc.)
df_clean['Weekday'] = df_clean['Start_Time'].dt.day_name()

# Calculate Duration (How long the accident lasted in minutes)
df_clean['Duration_Minutes'] = (df_clean['End_Time'] - df_clean['Start_Time']).dt.total_seconds() / 60

print("Created columns: 'Hour', 'Month', 'Weekday', 'Duration_Minutes'")


# 4. FINAL CLEANUP
# Requirement: Handle duplicate entries 
df_clean = df_clean.drop_duplicates()


# 5. VERIFY RESULTS
print("\n==============================")
print("WEEK 2 COMPLETION REPORT")
print("==============================")
print(f"Original Shape: {df.shape}")
print(f"Cleaned Shape:  {df_clean.shape}")
print("\nMissing Values Remaining (Top 7):")
print(df_clean.isnull().sum().sort_values(ascending=False).head(7))
print("\nNew Data Snapshot:")
display(df_clean[['Start_Time', 'Hour', 'Weekday', 'City', 'Severity']].head())

--- Cleaning Data ---
Dropped columns: ['End_Lat', 'End_Lng']
Filled missing Precipitation values with 0.
Dropped rows with missing City/Street info.

--- Converting Dates ---
Start_Time and End_Time converted to datetime objects.

--- Creating New Features ---
Created columns: 'Hour', 'Month', 'Weekday', 'Duration_Minutes'

WEEK 2 COMPLETION REPORT
Original Shape: (50000, 46)
Cleaned Shape:  (49993, 48)

Missing Values Remaining (Top 7):
Wind_Chill(F)        47496
Wind_Speed(mph)       8415
Humidity(%)            632
Weather_Condition      596
Visibility(mi)         519
Temperature(F)         449
Pressure(in)           299
dtype: int64

New Data Snapshot:


,Start_Time,Hour,Weekday,City,Severity
0,2016-02-08 05:46:00,5,Monday,Dayton,3
1,2016-02-08 06:07:59,6,Monday,Reynoldsburg,2
2,2016-02-08 06:49:27,6,Monday,Williamsburg,2
3,2016-02-08 07:23:34,7,Monday,Dayton,3
4,2016-02-08 07:39:07,7,Monday,Dayton,2
